In [ ]:
import os
os.environ['SPS_HOME'] = '/home/annalena/sps_fsps'

# Run once the original pipeline to get the rubixdata structure filled

In [ ]:
from rubix.core.pipeline import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 2,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run()

# Set target values and create targetdatacube

In [ ]:
import jax.numpy as jnp

rubixdata.stars.age = jnp.array([5.0, 5.0])
rubixdata.stars.metallicity = jnp.array([0.03, 0.03])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
#NBVAL_SKIP
from rubix.core.pipeline_gradient import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
     "data": {
        "args": {
            "particle_type": ["stars"],
        },
    },
    
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
    },        
}
pipe = RubixPipeline(config)
rubixdata = pipe.run(rubixdata)

target = rubixdata

# Define values at which the gradient will be applied

In [ ]:
import jax.numpy as jnp

rubixdata.stars.age = jnp.array([5.0, 5.0])
rubixdata.stars.metallicity = jnp.array([0.01, 0.01])
rubixdata.stars.mass = jnp.array([[1.0, 1.0]])
rubixdata.stars.velocity = jnp.array([[0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])

In [ ]:
from rubix.core.pipeline_gradient import RubixPipeline
# Suppose you already have a user_config or path to config
#config = "../rubix/config/pipeline_config.yaml"
import os
config = {
    "pipeline":{"name": "calc_gradient"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
     "data": {
        "args": {
            "particle_type": ["stars"],
        },
    },
    
    "output_path": "output",
    "output_modified":  False,

    "telescope":
        {"name": "TESTGRADIENT",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},
         },
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "FSPS"
        },
    },        
}

pipe = RubixPipeline(config)
loss = pipe.loss(rubixdata, target)
print(loss)
gradient = pipe.gradient(rubixdata, target)

In [ ]:
loss

In [ ]:
rubixdata.stars.age

In [ ]:
gradient.stars.age

In [ ]:
rubixdata

In [ ]:
from rubix.core.pipeline_gradient import RubixPipeline
import jax

metallicity_test = jnp.array([0.05, 0.05])
pipe = RubixPipeline(config)
loss = pipe.loss_only_wrt_metallicity(metallicity_test, rubixdata, target)
grad_stars_metallicity = jax.grad(pipe.loss_only_wrt_metallicity, argnums=0)(metallicity_test, rubixdata, target)
print(loss, grad_stars_metallicity)

In [ ]:
import jax
import jax.numpy as jnp
import numpy as np

# Initialize the pipeline
pipe = RubixPipeline(config)

# Prepare the data structure to store results
results = []

# Iterate over the age values from 1.0 to 10.3 in 0.1 steps
for metallicity in np.logspace(np.log10(0.5e-4), np.log10(0.05), 100):
    metallicity_test = jnp.array([metallicity, metallicity])
    loss = pipe.loss_only_wrt_age(metallicity_test, rubixdata, target)
    grad_stars_metallicity = jax.grad(pipe.loss_only_wrt_age, argnums=0)(metallicity_test, rubixdata, target)
    results.append((metallicity_test, loss, grad_stars_metallicity))

In [ ]:
# Save the results to a file
with open('./output/loss/gradient_wrt_metallicity_3e-2.txt', 'w') as f:
    for metallicity, loss, grad in results:
        f.write(f"Metallicity: {metallicity}, Loss: {loss}, Gradient: {grad}\n")

In [ ]:
metallicity = [result[0] for result in results]
losses = [result[1] for result in results]
print(metallicity)
print(losses)

In [ ]:
import matplotlib.pyplot as plt

metallicity = [result[0] for result in results]
losses = [result[1] for result in results]
gradients = [result[2] for result in results]

metallicity = jnp.array(metallicity)
losses = jnp.array(losses)
gradients = jnp.array(gradients)

#print(losses)
#print(gradients)

# Plot the gradient versus age
plt.figure(figsize=(10, 6))
plt.plot(metallicity[:, 0], gradients[:, 0], label='Gradient wrt metallicity')
plt.plot(metallicity[:, 0], losses, label='loss wrt metallicity')
plt.vlines(1.0e-4, -50, 50, colors='r', linestyles='dashed', label='True metallicity')
plt.xlabel('Metallicity')
plt.ylabel('Gradient')
plt.title('Gradient vs Metallicity')
plt.legend()
plt.grid(True)
#plt.xlim(0,0.0004)
#plt.ylim(0,1)
#plt.xscale('log')
plt.savefig('./output/loss/gradient_wrt_metallicity_3e-2.png')
plt.show()
plt.close()

In [ ]:
rubixdata.stars.metallicity